In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook


In [ ]:
import numpy as np

In [ ]:
from josie.bc import Dirichlet, Neumann, NeumannDirichlet
from josie.geom import Line
from josie.solver.euler import Q, EulerSolver, PerfectGas, Rusanov
from josie.mesh import Mesh, SimpleCell

left = Line([0, 0], [0, 1])
bottom = Line([0, 0], [1, 0])
right = Line([1, 0], [1, 1])
top = Line([0, 1], [1, 1])

eos = PerfectGas(gamma=1.4)

# Params
JET_CENTER = 0.5
JET_RADIUS = 0.05

# Inlet
U_JET = 1
V_JET = 0
RHO_JET = 1000
P_JET = 1e3

RHOe_JET = eos.rhoe(RHO_JET, P_JET)
E_JET = RHOe_JET/RHO_JET + 0.5 * (U_JET**2 + V_JET**2)
C_JET = eos.sound_velocity(RHO_JET, P_JET)
print(f"Mach: {U_JET/C_JET}")
Q_JET = Q(RHO_JET, RHO_JET*U_JET, RHO_JET*V_JET, RHO_JET*E_JET, RHOe_JET, U_JET, V_JET, P_JET, C_JET)

# Field conditions at init
U_INIT = 0
V_INIT = 0
RHO_INIT = RHO_JET/1000
P_INIT = P_JET
RHOe_INIT = eos.rhoe(RHO_INIT, P_INIT)
E_INIT = RHOe_INIT/RHO_INIT + 0.5 * (U_INIT**2 + V_INIT**2)
C_INIT = eos.sound_velocity(RHO_INIT, P_INIT)

print(f"Mach: {U_INIT/C_INIT}")

Q_INIT = Q(RHO_INIT, RHO_INIT*U_INIT, RHO_INIT*V_INIT, RHO_INIT*E_INIT, RHOe_INIT, 
           U_INIT, V_INIT, P_INIT, C_INIT)

# Neumann
dQ = Q.zeros()

def partition_fun(centroids: np.ndarray):
    yc = centroids[:, 1]
    
    # Partition cells of the inlet
    idx = np.where((yc - JET_CENTER)**2 < JET_RADIUS**2)
    
    return idx


# Assign BC to boundaries
left.bc = NeumannDirichlet(dirichlet_value=Q_JET, neumann_value=dQ, partition_fun=partition_fun)
# left.bc = Dirichlet(Q_JET)
top.bc = Dirichlet(Q_INIT)
right.bc = Neumann(dQ)
bottom.bc = Dirichlet(Q_INIT)

mesh = Mesh(left, bottom, right, top, SimpleCell)
mesh.interpolate(100, 100)
mesh.generate()
mesh.write('mesh.xdmf')
# mesh.plot()

Mach: 0.8451542547285166
Mach: 0.0


In [ ]:
# Solver
solver = EulerSolver(mesh, eos)



def init_fun(solver): 
    solver.values[:, :, :] = Q_INIT
    

solver.init(init_fun)

In [ ]:
final_time = 1
t = 0

t_save = 0
dt_save = 0.001
should_save = False


CFL = 0.2
scheme = Rusanov()

while t <= final_time:
    if should_save:
        solver.animate(t)
        solver.save(t, 'euler.xdmf')
        should_save = False
        
    dt = scheme.CFL(
        solver.values, 
        solver.mesh.volumes, 
        solver.mesh.normals,
        solver.mesh.surfaces,
        CFL,
    )
    
    cur_t = t + dt
    
    if cur_t > (t_save + dt_save):
        should_save = True
        dt = (t_save + dt_save) - t
        t_save = t + dt
    
    solver.step(dt, scheme)
    t += dt
    print(f't: {t}, dt: {dt}')



del solver._writer

t: 5.345224838248196e-05, dt: 5.345224838248196e-05
t: 0.00010548324920315425, dt: 5.20310008206723e-05
t: 0.00015751910992212084, dt: 5.203586071896658e-05
t: 0.00020955849192097768, dt: 5.203938199885684e-05
t: 0.00026160073501400595, dt: 5.2042243093028286e-05
t: 0.00031364532334535763, dt: 5.204458833135166e-05
t: 0.00036569186650681733, dt: 5.2046543161459694e-05
t: 0.00041774006270756483, dt: 5.20481962007475e-05
t: 0.0004697896741213881, dt: 5.204961141382324e-05
t: 0.0005218405101677654, dt: 5.205083604637736e-05
t: 0.0005738924158922822, dt: 5.20519057245168e-05
t: 0.0006259452636965841, dt: 5.205284780430189e-05
t: 0.0006779989472885087, dt: 5.2053683591924485e-05
t: 0.0007300533770706627, dt: 5.2054429782154086e-05
t: 0.0007821084764108357, dt: 5.205509934017295e-05
t: 0.0008341641784251314, dt: 5.205570201429569e-05
t: 0.0008862204230875853, dt: 5.2056244662453916e-05
t: 0.000938277154661083, dt: 5.205673157349771e-05
t: 0.0009903343196103437, dt: 5.205716494926056e-05
t: 0

t: 0.008103929756988473, dt: 5.1964904171845386e-05
t: 0.008155894765141366, dt: 5.19650081528922e-05
t: 0.008207859663416028, dt: 5.196489827466263e-05
t: 0.008259824430146036, dt: 5.196476673000681e-05
t: 0.008311789115775061, dt: 5.196468562902645e-05
t: 0.008363753768919915, dt: 5.196465314485466e-05
t: 0.008415718436413009, dt: 5.1964667493093465e-05
t: 0.008467683163350124, dt: 5.1964726937114087e-05
t: 0.00851964799314279, dt: 5.1964829792666016e-05
t: 0.00857161296757463, dt: 5.196497443184086e-05
t: 0.008623578048345877, dt: 5.1965080771246856e-05
t: 0.008675543090060934, dt: 5.1965041715055626e-05
t: 0.00872750813374973, dt: 5.196504368879748e-05
t: 0.008779473218765127, dt: 5.196508501539674e-05
t: 0.008831438382830303, dt: 5.196516406517528e-05
t: 0.00888340366208972, dt: 5.196527925941649e-05
t: 0.0089353690911631, dt: 5.196542907338115e-05
t: 0.008987334703201906, dt: 5.196561203880663e-05
t: 0.009000000000000001, dt: 1.2665296798095027e-05
t: 0.009051965834916487, dt: 5.

t: 0.016415964116096696, dt: 5.1995857487349815e-05
t: 0.01646796008349545, dt: 5.199596739875254e-05
t: 0.01651995616456805, dt: 5.199608107259886e-05
t: 0.016571952362960142, dt: 5.19961983920946e-05
t: 0.016623948661382903, dt: 5.1996298422762116e-05
t: 0.01667594506292371, dt: 5.199640154080734e-05
t: 0.016727941571426586, dt: 5.199650850287825e-05
t: 0.01677993819061711, dt: 5.199661919052279e-05
t: 0.016831934924104532, dt: 5.1996733487423775e-05
t: 0.01688393177538392, dt: 5.1996851279389626e-05
t: 0.0169359287478382, dt: 5.199697245427908e-05
t: 0.016987925844740247, dt: 5.199709690204859e-05
t: 0.017000000000000008, dt: 1.2074155259760716e-05
t: 0.0170519972549066, dt: 5.199725490659196e-05
t: 0.01710399462256244, dt: 5.199736765584072e-05
t: 0.017155992104098898, dt: 5.199748153645785e-05
t: 0.017207989702846736, dt: 5.1997598747837345e-05
t: 0.0172599874220289, dt: 5.199771918216334e-05
t: 0.01731198526476253, dt: 5.1997842733632454e-05
t: 0.017363983234060954, dt: 5.1997969

t: 0.024572166529821592, dt: 5.201546299975879e-05
t: 0.024624182055984746, dt: 5.2015526163154715e-05
t: 0.024676197644975476, dt: 5.201558899072885e-05
t: 0.024728213296459047, dt: 5.2015651483571404e-05
t: 0.024780229010101892, dt: 5.201571364284415e-05
t: 0.02483224478483499, dt: 5.2015774733095184e-05
t: 0.02488426061965142, dt: 5.201583481642966e-05
t: 0.02493627651422437, dt: 5.201589457295125e-05
t: 0.024988292468228295, dt: 5.201595400392344e-05
t: 0.025000000000000015, dt: 1.1707531771720098e-05
t: 0.025052016025276572, dt: 5.201602527655799e-05
t: 0.025104032108254286, dt: 5.2016082977714704e-05
t: 0.02515604824860748, dt: 5.201614035319353e-05
t: 0.02520806444601202, dt: 5.201619740454244e-05
t: 0.02526008070014538, dt: 5.201625413335767e-05
t: 0.025312097010686662, dt: 5.201631054128203e-05
t: 0.02536411337731667, dt: 5.2016366630010156e-05
t: 0.025416129799717954, dt: 5.201642240128309e-05
t: 0.025468146276494297, dt: 5.2016476776344314e-05
t: 0.025520162807099126, dt: 5.

t: 0.032884359724435985, dt: 5.202133188315596e-05
t: 0.0329363810773556, dt: 5.20213529196135e-05
t: 0.032988402451250204, dt: 5.202137389460449e-05
t: 0.03300000000000002, dt: 1.1597548749818498e-05
t: 0.033052021399459605, dt: 5.202139945958064e-05
t: 0.03310404281966345, dt: 5.202142020384065e-05
t: 0.03315606425971325, dt: 5.202144004979952e-05
t: 0.03320808571954993, dt: 5.2021459836683924e-05
t: 0.033260107199116075, dt: 5.202147956614615e-05
t: 0.03331212869835592, dt: 5.202149923984421e-05
t: 0.03336415021721536, dt: 5.202151885944173e-05
t: 0.03341617175564197, dt: 5.202153842660782e-05
t: 0.033468193313584985, dt: 5.202155794301702e-05
t: 0.03352021489099533, dt: 5.2021577410349096e-05
t: 0.03357223648782562, dt: 5.202159683028891e-05
t: 0.03362425810403014, dt: 5.2021616204526135e-05
t: 0.0336762797395649, dt: 5.2021635534755175e-05
t: 0.03372830139438757, dt: 5.2021654822674764e-05
t: 0.03378032306845756, dt: 5.202167406998788e-05
t: 0.03383234476173596, dt: 5.202169327840

...

t: 0.9526820440851825, dt: 5.684050014779781e-05
t: 0.9527388852203771, dt: 5.6841135194560394e-05
t: 0.9527957269906061, dt: 5.6841770229007913e-05
t: 0.9528525693958573, dt: 5.684240525112116e-05
t: 0.9529094124361182, dt: 5.6843040260880734e-05
t: 0.9529662561113764, dt: 5.684367525826704e-05
t: 0.9530000000000007, dt: 3.3743888624315765e-05
t: 0.953056844687177, dt: 5.68446871762626e-05
t: 0.9531136900093192, dt: 5.684532214218402e-05
t: 0.9531705359664147, dt: 5.6845957095459996e-05
t: 0.9532273825584509, dt: 5.6846592036143204e-05
t: 0.9532842297854152, dt: 5.6847226964257704e-05
t: 0.9533410776472949, dt: 5.684786187981067e-05
t: 0.9533979261440777, dt: 5.68484967827991e-05
t: 0.953454775275751, dt: 5.684913167321376e-05
t: 0.953511625042302, dt: 5.6849766551041735e-05
t: 0.9535684754437183, dt: 5.6850401416267704e-05
t: 0.9536253264799871, dt: 5.685103626887483e-05
t: 0.9536821781510959, dt: 5.685167110884538e-05
t: 0.953739030457032, dt: 5.6852305936160944e-05
t: 0.95379588339

t: 0.9620569449628931, dt: 5.694496289232822e-05
t: 0.9621138905585312, dt: 5.694559563812783e-05
t: 0.9621708367868993, dt: 5.694622836807233e-05
t: 0.9622277836479816, dt: 5.694686108221633e-05
t: 0.9622847311417622, dt: 5.6947493780585134e-05
t: 0.9623416792682253, dt: 5.694812646318659e-05
t: 0.9623986280273553, dt: 5.6948759130018264e-05
t: 0.9624555774191363, dt: 5.694939178107139e-05
t: 0.9625125274435526, dt: 5.695002441633338e-05
t: 0.9625694781005885, dt: 5.695065703578919e-05
t: 0.9626264293902279, dt: 5.695128963942238e-05
t: 0.9626833813124551, dt: 5.6951922227215456e-05
t: 0.9627403338672543, dt: 5.69525547991503e-05
t: 0.9627972870546095, dt: 5.6953187355208294e-05
t: 0.9628542408745049, dt: 5.695381989537059e-05
t: 0.9629111953269245, dt: 5.695445241961806e-05
t: 0.9629681504118525, dt: 5.695508492793146e-05
t: 0.9630000000000007, dt: 3.184958814828054e-05
t: 0.9630569560711022, dt: 5.695607110147689e-05
t: 0.9631139127746718, dt: 5.695670356965122e-05
t: 0.963170870110

t: 0.9715704750776791, dt: 5.705034243653148e-05
t: 0.9716275260499726, dt: 5.7050972293431075e-05
t: 0.9716845776521039, dt: 5.70516021313419e-05
t: 0.9717416298840542, dt: 5.7052231950246074e-05
t: 0.9717986827458044, dt: 5.705286175012543e-05
t: 0.9718557362373353, dt: 5.7053491530961404e-05
t: 0.9719127903586281, dt: 5.705412129273522e-05
t: 0.9719698451096636, dt: 5.705475103542793e-05
t: 0.9720000000000008, dt: 3.0154890337197315e-05
t: 0.9720570557135719, dt: 5.7055713571148576e-05
t: 0.9721141120568382, dt: 5.7056343266238165e-05
t: 0.9721711690297802, dt: 5.705697294197263e-05
t: 0.9722282266323786, dt: 5.705760259840796e-05
t: 0.9722852848646142, dt: 5.705823223557031e-05
t: 0.9723423437264677, dt: 5.705886185346817e-05
t: 0.9723994032179197, dt: 5.705949145209959e-05
t: 0.9724564633389512, dt: 5.7060121031456205e-05
t: 0.9725135240895427, dt: 5.7060750591525815e-05
t: 0.972570585469675, dt: 5.70613801322938e-05
t: 0.9726276474793287, dt: 5.706200965374405e-05
t: 0.9726847101

t: 0.9810571547074831, dt: 5.715470748237946e-05
t: 0.9811143100414016, dt: 5.715533391850577e-05
t: 0.9811714660017338, dt: 5.7155960332197835e-05
t: 0.9812286225884573, dt: 5.715658672351221e-05
t: 0.9812857798015499, dt: 5.7157213092475524e-05
t: 0.9813429376409889, dt: 5.715783943909658e-05
t: 0.9814000961067523, dt: 5.7158465763373715e-05
t: 0.9814572551988175, dt: 5.715909206529894e-05
t: 0.9815144149171624, dt: 5.7159718344860395e-05
t: 0.9815715752617644, dt: 5.716034460204381e-05
t: 0.9816287362326013, dt: 5.716097083683348e-05
t: 0.9816858978296504, dt: 5.716159704921269e-05
t: 0.9817430600528896, dt: 5.7162223239164065e-05
t: 0.9818002229022963, dt: 5.716284940666981e-05
t: 0.981857386377848, dt: 5.716347555171185e-05
t: 0.9819145504795223, dt: 5.7164101674271816e-05
t: 0.9819717152072965, dt: 5.716472777433128e-05
t: 0.9820000000000008, dt: 2.8284792704225126e-05
t: 0.9820571656636188, dt: 5.7165663617974516e-05
t: 0.9821143319532813, dt: 5.7166289662540855e-05
t: 0.9821714

t: 0.9909733853440584, dt: 5.7262941016487926e-05
t: 0.9910000000000008, dt: 2.6614655942336363e-05
t: 0.9910572638525319, dt: 5.726385253110206e-05
t: 0.9911145283273237, dt: 5.726447479185873e-05
t: 0.9911717934243506, dt: 5.726509702685122e-05
t: 0.9912290591435867, dt: 5.726571923613603e-05
t: 0.9912863254850065, dt: 5.7266341419739794e-05
t: 0.9913435924485842, dt: 5.726696357767153e-05
t: 0.9914008600342941, dt: 5.7267585709929815e-05
t: 0.9914581282421105, dt: 5.726820781650698e-05
t: 0.991515397072008, dt: 5.726882989739155e-05
t: 0.9915726665239606, dt: 5.7269451952569726e-05
t: 0.9916299365979426, dt: 5.7270073982026226e-05
t: 0.9916872072939283, dt: 5.727069598574478e-05
t: 0.9917444786118921, dt: 5.7271317963708546e-05
t: 0.991801750551808, dt: 5.727193991590023e-05
t: 0.9918590231136503, dt: 5.7272561842302246e-05
t: 0.9919162962973932, dt: 5.7273183742896785e-05
t: 0.9919735701030109, dt: 5.72738056176659e-05
t: 0.9920000000000008, dt: 2.6429896989865753e-05
t: 0.99205727